Sample of code below.

In [ ]:
import pandas as pd

main_file_path = '../input/train.csv'
data_train = pd.read_csv(main_file_path)
# if i want to see columns
print(data_train.columns)


In [ ]:
data_train
#LotShape,SaleType,SaleCondition,

In [ ]:
col_interest =['ScreenPorch','MoSold','LotShape','SaleType','SaleCondition']
sa= data_train[col_interest]
sa.describe()

In [ ]:
#what we actually want to predict
y= data_train.SalePrice


In [ ]:
predicators =['YearBuilt','YrSold','TotalBsmtSF','LotShape','SaleType','SaleCondition']
one_hot_encoded_training_predictors = pd.get_dummies(predicators)
# we are using pd.get_dummies for encoding .
one_hot_encoded_training_predictors
#predicators.dtypes.sample(10)

In [ ]:
X = data_train[predicators]
X


****Sklearn is the most popular library for modelling the types of data in dataframe.

Define: What type of model? Decision tree? Naive bayes? KNN? **

Fit :Capture patterns from provided data..This is heart of modelling.

Predict: do it.

Evaluate: Determine how accurate the model is.**


In [ ]:
from sklearn.tree import DecisionTreeRegressor
# Define the model
housing_model = DecisionTreeRegressor()

#Fit model
housing_model.fit(X,y)



In [ ]:
#Lets make prediction on he houses in the training dataset.
print(" making predictions for the following 5 houses:")
print(X.head())
print ("The prediction are")
print(housing_model.predict(X.head()))

In [ ]:
# here we use MAE( Mean Absolute error)
from sklearn.metrics import mean_absolute_error
predicted_Home_prices = housing_model.predict(X)
mean_absolute_error(y,predicted_Home_prices)

In [ ]:
# use the data that was not used to build the model. This is the validation data.
# scikit learn library has a function train_test_split data into two pieces.
from sklearn.model_selection import train_test_split
#split data into tarining and validation data, for both predicators and target
# use random_state is used to provide the same split every time we run this script.
train_X,val_X,train_y,val_y = train_test_split(X,y,random_state=0)
#Define Model
housing_model = DecisionTreeRegressor()
#Fit model
housing_model.fit(train_X,train_y)

#get predicted prices on validation data
val_predictions =housing_model.predict(val_X)
print(mean_absolute_error(val_y,val_predictions))




**Experimenting with different models**
Overfitting: Where a model matches the training data almost perfectly, but does poorly in validation and other new data.
Underfitting: When model fails to capture important distinctions and patterns in the data, so it performs poorly even in training data, that is underfitting.

In [ ]:
# we want to compare the Mean absolute error(MAE) from different values for max_leaf_nodes.
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeRegressor

def get_mae(max_leaf_nodes,predictors_train,predictors_val,targ_train,targ_val):
 # Define the model  
    model=DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=0)
# fit the model. fit the data.
    model.fit(predictors_train,targ_train)
# predict the values    
    preds_val =model.predict(predictors_val)
# calculate Mean absolute error
    mae =mean_absolute_error(targ_val,preds_val)
    return(mae)


In [ ]:
# comapre the MAE with different values of max_leaf_nodes
for max_leaf_nodes in [5,50,500,5000]:
    my_mae = get_mae(max_leaf_nodes,train_X,val_X,train_y,val_y)
    print("Max leaf nodes :%d  \t\t  Mean Absolute Error: %d" %(max_leaf_nodes, my_mae))
# learn to format string
          #http://www.diveintopython.net/native_data_types/formatting_strings.html
          # Trying to concatenate a string with a non-string raises an exception so we
          #have to use % to convert it int into string.
          


**Learning more sophistiated algorithms know as "Random Forest"** because Decision tree is not a sophisticated model. Generally we do not get validation data in model training

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
#define the model
forest_model =RandomForestRegressor()
# fit the model with the data. it is the training data
forest_model.fit(train_X,train_y)
#predict 
predict_vals = forest_model.predict(val_X)
# calculate the Mean absolute error
print(mean_absolute_error(val_y,predict_vals))


It is a improvment from 42k to 34k.

In [ ]:
# We are learning to make submission. here we have a separate file known as test.
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor

#reading the training file
train = pd.read_csv('../input/train.csv')

# pull data into target and predictors.
train_y= train.SalePrice
predictor_cols =['LotArea','OverallQual','YearBuilt','TotRmsAbvGrd']
# Create training predictors data
train_X =train[predictor_cols]
# define model
my_model = RandomForestRegressor()
# fit the model
my_model.fit(train_X,train_y)


In [ ]:
#Read the test file
test = pd.read_csv('../input/test.csv')
# test is like train data except no predictor value.That is what is our task
test_X=test[predictor_cols]
#now please use the model that was created in previous step to predict the outcome
predict_prices =my_model.predict(test_X)
print(predict_prices)
# difference between print(predict_prices) and predict_prices is [....]  and ([....])
# it means immutable that is tuple and mutable that is List.
#predict_prices

# remember we cannot have Mean absolute error here because we dont have a predicted values.


**PREPARE SUBMISSION FILE**

In [ ]:
my_submission = pd.DataFrame({'Id': test.Id, 'SalePrice': predict_prices})
# it is a dictionary {}, a key value pair
my_submission.to_csv('submission.csv',index=False)

In [ ]:
# handling missing values
#Option one is to drop column with missing values.


In [ ]:
import pandas as pd
train_data =pd.read_csv('../input/train.csv')
test_data =pd.read_csv('../input/test.csv')

In [ ]:
# we will not include houses in training dataset where houses are missing.
train_data.dropna(axis=0,subset=['SalePrice'],inplace=True)
target =train_data.SalePrice
# we will drop the missingvalues columns which actually is not the right approach, we 
#could have have used the imputed method .
cols_with_missing =[col for col in train_data.columns
                               if train_data[col].isnull().any()]
# drop these columns and also drop Id and SalePrice
candidate_train_predictors = train_data.drop(['Id','SalePrice'] + cols_with_missing,axis=1)
candidate_test_predictors = test_data.drop(['Id']+ cols_with_missing,axis=1)

# cardinality means the number of unique values in a column
#We use it as our only way to select categorical columns here.Ha ha ha. We select the low cardinality 
#column as a indicator of categorical values.
low_cardinality_cols = [cname for cname in candidate_train_predictors.columns if 
                              candidate_train_predictors[cname].nunique() <10 and 
                              candidate_train_predictors[cname].dtype =="object"]                         
numeric_cols =[cname for cname in candidate_train_predictors.columns if
                     candidate_train_predictors[cname].dtype in ['int64','float64']]
my_cols = low_cardinality_cols + numeric_cols
train_predictors =candidate_train_predictors[my_cols]
test_predictors = candidate_test_predictors[my_cols]

train_predictors.dtypes.sample(10)



In [ ]:
one_hot_encoded_training_predictors =pd.get_dummies(train_predictors)
one_hot_encoded_training_predictors

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor

def get_mae(X,y):
    # multiple by -1 to make positive MAE score instead of neg value returned as sklearn convention
    return -1 * cross_val_score(RandomForestRegressor(50), X,y, scoring='neg_mean_absolute_error').mean()

predictors_without_categoricals = train_predictors.select_dtypes(exclude=['object'])

mae_without_categoricals =get_mae(one_hot_encoded_training_predictors,target)
mae_one_hot_encoded = get_mae(one_hot_encoded_training_predictors,target)
print('Mean Absolute Error when Dropping Categoricals:' +str(int(mae_without_categoricals)))
print('Mean Absolute Error with One-Hot Encoding:' +str(int(mae_one_hot_encoded)))


In [ ]:
# the challenge is if training and test gets misalinged, results will be nonsense.
# use align command 
one_hot_encoded_training_predictors =pd.get_dummies(train_predictors)
one_hot_encoded_test_predictors =pd.get_dummies(test_predictors)
final_train,final_test = one_hot_encoded_training_predictors.align(one_hot_encoded_test_predictors, 
                                                                   join='left',axis=1)
final_train
final_test

#Gradient Boosting with XGBoost
XGBoost is the leading model for working with standard tabular data (it is the type of data stored in Pandas DataFrames, and not data like images and videos)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Imputer

data_train =pd.read_csv('../input/train.csv')
data_train.dropna(axis=0,subset=['SalePrice'],inplace=True)
y=data_train.SalePrice
X=data_train.drop(['SalePrice'], axis=1).select_dtypes(exclude=['object'])
# we are converting it to matrix
train_X,test_X,train_Y,test_y=train_test_split(X.as_matrix(),y.as_matrix(),test_size=0.25)
#label  = X[:,32]
#weights = np.ones(len(labels))
#dtrain = xgb.DMatrix(X, label = labels, weight = weights)
# using imputer function of sklearn
my_imputer = Imputer()
# fit the model
train_X =my_imputer.fit_transform(train_X)
test_X =my_imputer.transform(test_X)
    

In [ ]:
from xgboost import XGBRegressor
#Define the model
my_model =XGBRegressor()
my_model.fit(train_X,train_Y,verbose=False)

In [ ]:
# make predictions
predictions= my_model.predict(test_X)
from sklearn.metrics import mean_absolute_error
print("Mean absolute error :" + str(mean_absolute_error(predictions,test_y)))
#train_X,test_X,train_Y,test_y

In [ ]:
#n_estimators and early_stopping_rounds
my_model = XGBRegressor(n_estimators=1000,learning_rate=0.5)
my_model.fit(train_X, train_y, early_stopping_rounds=5, 
             eval_set=[(test_X, test_y)], verbose=False)


In [ ]:
data_train.columns

In [ ]:
def get_some_data():
    cols_to_use =['LotArea','TotalBsmtSF','TotRmsAbvGrd']
    data =pd.read_csv('../input/train.csv')
    Y=data.SalePrice
    x=data[cols_to_use]
    my_imputer =Imputer()
    imputed_X =my_imputer.fit_transform(X)
    return imputed_X,y

#partisl dependence to figure out complex patterns and dependences
from sklearn.ensemble.partial_dependence import partial_dependence,plot_partial_dependence
from sklearn.ensemble import GradientBoostingRegressor
#from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
# get_some_data is defined in hidden cell above
X,y = get_some_data()
#define
my_model =GradientBoostingRegressor()

#fit the model
my_model.fit(X,y)
# Here we make the plot
my_plots = plot_partial_dependence(my_model,features=[1,2], #column number of plots we want to show
                                  X=X, #raw predictor data
                                  feature_names=['LotArea','TotalBsmtSF','TotRmsAbvGrd'],# labels on graphs
                                  grid_resolution =10)# number of values to plot on X axis


# What are Pipelines?
Pipelines bind preprocessing and modelling, so that whole bundle can be used as if it were a single step
1)Cleaner Code: no need to take care of training and validation data at each step of processing.
2)the challenge is, it is tough to transition a model from prototype to something deployable at scale. 
3)helps in cross validation

In [ ]:
# let load the data and set predicators and divide it into train and test
import pandas as pd
from sklearn.model_selection import train_test_split

#Read data
data= pd.read_csv('../input/train.csv')
cols_to_use =['BedroomAbvGr','LotArea','PoolArea','TotRmsAbvGrd','YrSold']
X= data[cols_to_use]
y=data.SalePrice
train_X,test_X,train_y,test_y = train_test_split(X,y)
#Modelling proces that will fill Imputer at missing places,
#than folowed by RandomForestRegressor to make predictions
# will bundle Imputer and RandomForestRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocesing import Imputer
# pipelines binds pre processing and modeling
my_pipeline =make_pipeline(Imputer(),RandomForestRegressor())
#now predict as a fued whole
#fit
my_pipeline.fit(train_X,train_y)
#predictions
predictions =my_pipeline.predict(test_X)

# Sciket learn are of two categories. Transformers and models
# Transformers are for pre-processing and models are for predictions
# generally after fitting a transformer we apply transform command,After fitting a model we apply predict
#command









**CROSS VALIDATION**




In [ ]:
import pandas as pd
data_train=pd.read_csv('../input/train.csv')
cols_to_use =['LotArea','OverallQual','YearBuilt','TotRmsAbvGrd']
X =data_train[cols_to_use]
y= data_train.SalePrice

#NOTE: IT IS EASY TO DO CROSS VALIDATON IF PIPELINE IS BEING USED
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Imputer
my_pipeline = make_pipeline(Imputer(),RandomForestRegressor())

#lets do the cross validation

from sklearn.model_selection import cross_val_score
scores = cross_val_score(my_pipeline,X,y,scoring='neg_mean_absolute_error')
print(scores)
# take the average of the mean 
print('Mean Absolute Error %2f'%(-1 * scores.mean()))

What is Data Leakage?
Leakage causes a model to look acccurate until you start making decisions with the model.there are two main types of leakages:
**Leaky Predictors** and **Leaky validation Strategies.**




In [ ]:
#Leaky Predictors
#This occurs when your predictors include data that will not be available at the time you make predictions
# Leaky Validation Strategy
#A much different type of leak occurs when you aren't careful distinguishing training data from validation data.
#Your model will get very good validation scores, giving you great confidence in it,
#but perform poorly when you deploy it to make decisions.
# The biggest challenge is that leaky predictors have HIGH CORRELATION WITH THE TARGET.

